In [1]:
# Import libraries
from datetime import timedelta
import os

import numpy as np
import pandas as pd
import re
import matplotlib.pyplot as plt
import matplotlib.dates as mdates

from IPython.display import display, HTML, Image
%matplotlib inline

plt.style.use('ggplot')
plt.rcParams.update({'font.size': 20})

# Access data using Google BigQuery.
from google.colab import auth
from google.cloud import bigquery

In [2]:
# authenticate
auth.authenticate_user()

In [3]:
# Set up environment variables
project_id = 'project-mimic-430923'
if project_id == 'CHANGE-ME':
  raise ValueError('You must change project_id to your GCP project.')
os.environ["GOOGLE_CLOUD_PROJECT"] = project_id

# Read data from BigQuery into pandas dataframes.
def run_query(query, project_id=project_id):
  return pd.io.gbq.read_gbq(
      query,
      project_id=project_id,
      dialect='standard')

# set the dataset
# if you want to use the demo, change this to mimic_demo
dataset = 'mimiciv'

In [105]:
# 1. Get basic patient info (admissions + demographics (excluding marital_status) + death status)
patient_info_query = """
SELECT adm.subject_id, adm.hadm_id, adm.admittime, adm.dischtime, pat.gender, pat.anchor_age AS age, adm.race, adm.hospital_expire_flag
FROM `physionet-data.mimiciv_hosp.admissions` AS adm
JOIN `physionet-data.mimiciv_hosp.patients` AS pat
ON adm.subject_id = pat.subject_id
WHERE adm.admittime IS NOT NULL
"""
patient_info_df = run_query(patient_info_query)

patient_info_df['in_hospital_death'] = patient_info_df['hospital_expire_flag'] == 1

patient_info_df = patient_info_df.drop(columns=['hospital_expire_flag'])

<ipython-input-3-d70a68234fd1>:9: FutureWarning: read_gbq is deprecated and will be removed in a future version. Please use pandas_gbq.read_gbq instead: https://pandas-gbq.readthedocs.io/en/latest/api.html#pandas_gbq.read_gbq
  return pd.io.gbq.read_gbq(


In [106]:
patient_info_df

,subject_id,hadm_id,admittime,dischtime,gender,age,race,in_hospital_death
0,14962874,22614141,2165-10-15 15:59:00,2165-10-18 16:47:00,M,89,WHITE,False
1,13241600,22574379,2162-10-29 15:36:00,2162-10-30 16:00:00,M,89,WHITE,False
2,12945423,20185299,2122-07-08 05:40:00,2122-07-19 15:15:00,M,89,WHITE,False
3,16014534,21633959,2176-03-03 14:33:00,2176-03-11 13:57:00,F,89,WHITE,False
4,16014534,27188276,2176-03-27 11:53:00,2176-03-31 14:10:00,F,89,WHITE,False
...,...,...,...,...,...,...,...,...
431226,12545126,27135177,2187-11-01 15:47:00,2187-11-02 09:45:00,M,91,WHITE,False
431227,17579295,26848807,2162-11-16 07:15:00,2162-11-19 14:27:00,M,91,WHITE,False
431228,18563244,26485584,2136-08-18 14:48:00,2136-08-19 11:00:00,F,91,WHITE,False
431229,14865704,29199399,2156-11-12 13:59:00,2156-11-13 14:00:00,F,91,OTHER,False


In [107]:
# Count number of patients
num_patients = patient_info_df['subject_id'].nunique()
print(f"Number of patients: {num_patients}")

Number of patients: 180733


In [84]:
ap_icd_query = f"""
SELECT subject_id, hadm_id, icd_code
FROM `physionet-data.mimiciv_hosp.diagnoses_icd`
WHERE icd_code LIKE 'K85%'
ORDER BY subject_id
"""
ap_icd_df = run_query(ap_icd_query)

<ipython-input-3-d70a68234fd1>:9: FutureWarning: read_gbq is deprecated and will be removed in a future version. Please use pandas_gbq.read_gbq instead: https://pandas-gbq.readthedocs.io/en/latest/api.html#pandas_gbq.read_gbq
  return pd.io.gbq.read_gbq(


In [85]:
ap_icd_df

,subject_id,hadm_id,icd_code
0,10002807,28464737,K8590
1,10004606,29242151,K8510
2,10006431,24638489,K8580
3,10008816,22267961,K859
4,10012206,23961896,K8510
...,...,...,...
1872,19939903,26616095,K851
1873,19968351,23732375,K8590
1874,19972266,24170910,K8590
1875,19990545,23106222,K8590


In [87]:
ap_icd_df['subject_id'].nunique()

1401

In [109]:
#merged_ap_icd_df = pd.merge(patient_info_df, ap_icd_df, on=['subject_id', 'hadm_id'], how='inner')

# Calculate length of stay
patient_info_df['admittime'] = pd.to_datetime(patient_info_df['admittime'])
patient_info_df['dischtime'] = pd.to_datetime(patient_info_df['dischtime'])
patient_info_df['length_of_stay'] = (patient_info_df['dischtime'] - patient_info_df['admittime']).dt.total_seconds() / (60 * 60 * 24)

# Merge length of stay into the dataset
merged_ap_icd_df = pd.merge(patient_info_df, ap_icd_df[['subject_id', 'hadm_id']], on=['subject_id', 'hadm_id'], how='right')

In [110]:
merged_ap_icd_df

,subject_id,hadm_id,admittime,dischtime,gender,age,race,in_hospital_death,length_of_stay
0,10002807,28464737,2152-03-30 16:09:00,2152-03-31 08:20:00,M,57,BLACK/AFRICAN AMERICAN,False,0.674306
1,10004606,29242151,2159-02-20 13:43:00,2159-03-06 16:51:00,F,64,WHITE,False,14.130556
2,10006431,24638489,2129-01-24 01:08:00,2129-01-30 16:50:00,F,66,WHITE,False,6.654167
3,10008816,22267961,2129-04-03 22:21:00,2129-04-04 06:45:00,M,36,WHITE,False,0.350000
4,10012206,23961896,2127-07-04 01:16:00,2127-07-14 17:00:00,M,45,WHITE,False,10.655556
...,...,...,...,...,...,...,...,...,...
1872,19939903,26616095,2165-03-13 21:33:00,2165-03-19 15:18:00,M,80,WHITE,False,5.739583
1873,19968351,23732375,2166-09-06 10:02:00,2166-09-07 18:08:00,F,66,WHITE,False,1.337500
1874,19972266,24170910,2177-01-28 08:44:00,2177-01-29 11:06:00,M,43,WHITE,False,1.098611
1875,19990545,23106222,2139-10-04 23:11:00,2139-10-29 15:45:00,F,43,ASIAN - ASIAN INDIAN,False,24.690278


In [111]:
# Count the number of patients with high lipase levels who had a CT scan
num_ap_icd_df = merged_ap_icd_df['subject_id'].nunique()
print(f"Number of patients with high lipase levels who had a CT scan: {num_ap_icd_df}")

# Count the number of rows in the dataset
num_rows = merged_ap_icd_df.shape[0]
print(f"Number of rows in the dataset: {num_rows}")

Number of patients with high lipase levels who had a CT scan: 1401
Number of rows in the dataset: 1877


In [112]:
def select_row(group):
    # Check if there's any row with hospital_expire_flag = 1
    if (group['in_hospital_death']).any():
        # Return the first row with hospital_expire_flag = 1
        return group[group['in_hospital_death']].iloc[0]
    else:
        # Otherwise, return the row with the maximum length_of_stay
        return group.loc[group['length_of_stay'].idxmax()]

# Apply the function to each group and reset the index
ap_filtered_df = merged_ap_icd_df.groupby(['subject_id']).apply(select_row).reset_index(drop=True)

<ipython-input-112-c3d881ddbe33>:11: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  ap_filtered_df = merged_ap_icd_df.groupby(['subject_id']).apply(select_row).reset_index(drop=True)


In [117]:
ap_filtered_df

,subject_id,hadm_id,admittime,dischtime,gender,age,race,in_hospital_death,length_of_stay
0,10002807,28464737,2152-03-30 16:09:00,2152-03-31 08:20:00,M,57,BLACK/AFRICAN AMERICAN,False,0.674306
1,10004606,29242151,2159-02-20 13:43:00,2159-03-06 16:51:00,F,64,WHITE,False,14.130556
2,10006431,24638489,2129-01-24 01:08:00,2129-01-30 16:50:00,F,66,WHITE,False,6.654167
3,10008816,22267961,2129-04-03 22:21:00,2129-04-04 06:45:00,M,36,WHITE,False,0.350000
4,10012206,23961896,2127-07-04 01:16:00,2127-07-14 17:00:00,M,45,WHITE,False,10.655556
...,...,...,...,...,...,...,...,...,...
1396,19939903,26616095,2165-03-13 21:33:00,2165-03-19 15:18:00,M,80,WHITE,False,5.739583
1397,19968351,23732375,2166-09-06 10:02:00,2166-09-07 18:08:00,F,66,WHITE,False,1.337500
1398,19972266,24170910,2177-01-28 08:44:00,2177-01-29 11:06:00,M,43,WHITE,False,1.098611
1399,19990545,23106222,2139-10-04 23:11:00,2139-10-29 15:45:00,F,43,ASIAN - ASIAN INDIAN,False,24.690278


In [128]:
ap_filtered_df.to_csv('AP_ICD_Dataset.csv', encoding='utf-8', index = False)

In [114]:
# 5. Retrieve comorbid conditions (ICD codes)
hadm_ids = ', '.join(str(hadm_id) for hadm_id in ap_filtered_df['hadm_id'].unique())
comorbidity_query = f"""
SELECT subject_id, hadm_id, icd_code, seq_num, icd_version
FROM `physionet-data.mimiciv_hosp.diagnoses_icd`
WHERE hadm_id IN ({hadm_ids})
ORDER BY subject_id, hadm_id, seq_num
"""
comorbidities_df = run_query(comorbidity_query)

# Merge comorbidities with the dataset
merged_comorbidities_df = pd.merge(ap_filtered_df, comorbidities_df, on=['subject_id', 'hadm_id'], how='left')

# Display the final merged dataset
print(merged_comorbidities_df.head())

<ipython-input-3-d70a68234fd1>:9: FutureWarning: read_gbq is deprecated and will be removed in a future version. Please use pandas_gbq.read_gbq instead: https://pandas-gbq.readthedocs.io/en/latest/api.html#pandas_gbq.read_gbq
  return pd.io.gbq.read_gbq(


   subject_id   hadm_id           admittime           dischtime gender  age  \
0    10002807  28464737 2152-03-30 16:09:00 2152-03-31 08:20:00      M   57   
1    10002807  28464737 2152-03-30 16:09:00 2152-03-31 08:20:00      M   57   
2    10002807  28464737 2152-03-30 16:09:00 2152-03-31 08:20:00      M   57   
3    10002807  28464737 2152-03-30 16:09:00 2152-03-31 08:20:00      M   57   
4    10002807  28464737 2152-03-30 16:09:00 2152-03-31 08:20:00      M   57   

                     race  in_hospital_death  length_of_stay icd_code  \
0  BLACK/AFRICAN AMERICAN              False        0.674306    K8590   
1  BLACK/AFRICAN AMERICAN              False        0.674306    I2510   
2  BLACK/AFRICAN AMERICAN              False        0.674306      I10   
3  BLACK/AFRICAN AMERICAN              False        0.674306     E785   
4  BLACK/AFRICAN AMERICAN              False        0.674306     K219   

   seq_num  icd_version  
0        1           10  
1        2           10  
2       

In [133]:
merged_comorbidities_df.to_csv('AP_ICD_Comorbidities_Dataset.csv', encoding='utf-8', index = False)

In [115]:
# Count the number of unique patients who died in the hospital
num_patients_with_death_info = merged_comorbidities_df[merged_comorbidities_df['in_hospital_death'] == True][['subject_id', 'hadm_id']].drop_duplicates().shape[0]
print(f"Number of unique patients who died in the hospital: {num_patients_with_death_info}")

Number of unique patients who died in the hospital: 44


In [118]:
# 1. Handle missing values
missing_data = merged_comorbidities_df.isnull().sum()
print(missing_data)

subject_id           0
hadm_id              0
admittime            0
dischtime            0
gender               0
age                  0
race                 0
in_hospital_death    0
length_of_stay       0
icd_code             0
seq_num              0
icd_version          0
dtype: int64
